# Teste do SymSpellPy

### Objetivo:

Analizar e comparar os tempos que o [SymSpellPy](https://symspellpy.readthedocs.io/en/latest/index.html) e o Spellchecker da Pre-Processing realizam a mesma atividade para assim ver se é possível a substituição do Spellchecker que é utilizado no laboratório (escrito em Go) por um que tenha sido escrito em Python.

In [1]:
from symspellpy import SymSpell, Verbosity
from pre_processing import CorpusHandler as ch
import time
import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/matheus/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/matheus/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Funções para automatizar o trabalho

## Funções com SymSpellPy

In [2]:
def spellPy(wrong_string):

    sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

    dictionary_path = "/home/matheus/Documents/UNB/AILab/spellcheker/vocabulary.txt"

    if not sym_spell.create_dictionary(dictionary_path):
        print("dictionary_path invalid!")
        return


    print("Number of words in the dictionary {}".format(sym_spell.word_count))

    print(type(wrong_string))
    if type(wrong_string) == list:
        initialTime = time.process_time()
        spellList(wrong_string, sym_spell)
        return time.process_time() - initialTime
    elif (len(wrong_string.split(' '))) == 1:
        initialTime = time.process_time()
        spellWord(wrong_string, sym_spell)
        return time.process_time() - initialTime
    else:
        initialTime = time.process_time()
        spellPhrase(wrong_string, sym_spell)
        return time.process_time() - initialTime



def spellWord(input_term, sym_spell):
    suggestions = sym_spell.lookup(input_term, Verbosity.CLOSEST, 2)

    for suggestion in suggestions:
        print("{} {} {}".format(suggestion.term,suggestion.distance, suggestion.count))

def spellPhrase(input_term, sym_spell):
    suggestions = sym_spell.lookup_compound(input_term,2)
   

def spellList(input_term, sym_spell):
    for i in input_term:
        spellPhrase(i,sym_spell)

## Funções com o Spellchecker da Pre-Prrocessing

In [3]:
def preProcessing(text):
        initial_time = time.process_time()

        result = ch.spellchecker_batch([(0,text)])


        # print(result)
        return time.process_time() - initial_time

def preProcess_list(text):
    initial_time = time.process_time()
    result = ch.spellchecker_batch(list(enumerate(text)))

    # print(result)

    return time.process_time() - initial_time


# Teste com apenas uma string grande

In [4]:
time_symspellpy = spellPy("""Teste com uma string bem grande""" * 100)
print(time_symspellpy)

Number of words in the dictionary 258463
<class 'str'>
9.909895822000001


In [5]:
time_pre_processing = preProcessing("""Teste com uma string bem grande""" * 100)
print(time_pre_processing)

Running spellchecker: 100%|██████████| 1/1 [00:00<00:00, 58.61texts/s]

0.03455803299999971


### Analisando os tempos

Tabela para melhor vizualização

In [6]:
table = pd.DataFrame(data=[ [time_symspellpy , time_pre_processing ]], columns=["symSpellPy", "Pre-Processing"],index=["Tempo"])
table

,symSpellPy,Pre-Processing
Tempo,9.909896,0.034558


O tempo usado pela lib SymSpellPy é aproximadamente 332.76 vezes o tempo que o Pre-Processing usou

In [7]:
time_symspellpy/time_pre_processing

286.7609919233564

# Comparação usando várias strings pequenas

In [8]:
text = [
    "começo da lista de textos",
    "essa parte é pato",
    "hoje tem gol do Gabigol",
    "Espero que dê tudo certo",
    "Tenho de terminar essa tarefa logo",
    "porém a pressa é inimiga da perfeição",
    "Logo, acho que eu deveria demorar bastante",
    "não é possivel demorar",
    "irei logo mais ir mostrar os resultados",
    "antes de mais nada",
    "fim da lista"
]*50

In [9]:
time_symspell_list = spellPy(text)
time_symspell_list

Number of words in the dictionary 258463
<class 'list'>


3.176298054

In [10]:
time_pre_processing_list = preProcess_list(text)
time_pre_processing_list

Running spellchecker: 100%|██████████| 550/550 [00:00<00:00, 3117.16texts/s]


0.2444506120000014

### Analisando os tempos

Tabela para melhor vizualização

In [11]:
table = pd.DataFrame(data=[ [time_symspell_list , time_pre_processing_list]], columns=["symSpellPy", "Pre-Processing"],index=["Tempo"])
table

,symSpellPy,Pre-Processing
Tempo,3.176298,0.244451


O tempo usado pela lib do SymSpellPy levou 13.56 a mais do que a Pre-Processing levou

In [12]:
time_symspell_list/time_pre_processing_list

12.993618743732096

## Conclusão

Concluimos que é inviável a utilização da SymSpellPy no laboratório do AI.Lab, pois apesar dela ser escrita em Python e isso trazer uma facilidade para as pessoas que pesquisam no laboratório, tendo em vista o grande uso da linguagem por parte da equipe, ela se mostrou muito lenta quando comparada com a sua concorrente escrita em Go.